In [1]:
import sqlite3
import requests
import time

import os

In [2]:

WB_TOKEN = os.getenv('WB_TOKEN')
DB_NAME = os.getenv('DB_NAME')

In [3]:
# Параметры подключения к базе
DB_PATH = "ue.db"
WB_API_URL = "https://discounts-prices-api.wildberries.ru/api/v2/upload/task"
HEADERS = {"Authorization": f"Bearer {WB_TOKEN}", "Content-Type": "application/json"}

# Подключение к базе и выборка записей
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

query = """
    SELECT nm_id, new_price, new_discount
    FROM products
    WHERE available_count = 5
"""
cursor.execute(query)
rows = cursor.fetchall()
conn.close()

# Формируем список товаров для обновления цен и скидок
goods_data = [{"nmID": nm_id, "price": new_price, "discount": new_discount} for nm_id, new_price, new_discount in rows]

# Отправка данных батчами по 1000 товаров (API поддерживает максимум 1000 за запрос)
BATCH_SIZE = 1000
REQUEST_LIMIT = 10  # Максимум 10 запросов за 6 секунд
REQUEST_INTERVAL = 6 / REQUEST_LIMIT  # Интервал между запросами

for i in range(0, len(goods_data), BATCH_SIZE):
    batch = goods_data[i:i + BATCH_SIZE]

    payload = {"data": batch}
    response = requests.post(WB_API_URL, json=payload, headers=HEADERS)

    if response.status_code == 200:
        print(f"✅ Успешно обновлены цены и скидки для {len(batch)} товаров")
    else:
        print(f"❌ Ошибка при обновлении: {response.text}")

    time.sleep(REQUEST_INTERVAL)  # Задержка для соблюдения лимита API


✅ Успешно обновлены цены и скидки для 81 товаров


In [4]:

WB_API_N_URL = "https://marketplace-api.wildberries.ru/api/v3/stocks/{warehouseId}"
WAREHOUSE_ID = 1283008


In [5]:
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

query = """
    SELECT skus, pcs, available_count
    FROM products
    WHERE skus IS NOT NULL
"""
cursor.execute(query)
rows = cursor.fetchall()
conn.close()

# Формируем список остатков
stocks_data = [
    {
        "sku": skus,
        "amount": (
            1 if available_count == 5 and pcs == 50 else
            2 if available_count == 5 and pcs == 30 else
            5 if available_count == 5 and pcs == 10 else
            0
        )
    }
    for skus, pcs, available_count in rows
]

In [6]:
stocks_data

[{'sku': '2042698806296', 'amount': 2},
 {'sku': '2042698762264', 'amount': 2},
 {'sku': '2042698762332', 'amount': 5},
 {'sku': '2042698762387', 'amount': 2},
 {'sku': '2042698762431', 'amount': 2},
 {'sku': '2042698762592', 'amount': 5},
 {'sku': '2042698762745', 'amount': 1},
 {'sku': '2042698762738', 'amount': 2},
 {'sku': '2042698762844', 'amount': 2},
 {'sku': '2042698762905', 'amount': 2},
 {'sku': '2042698762943', 'amount': 2},
 {'sku': '2042698765661', 'amount': 5},
 {'sku': '2042698765876', 'amount': 1},
 {'sku': '2042698766095', 'amount': 1},
 {'sku': '2042698766194', 'amount': 1},
 {'sku': '2042698766224', 'amount': 5},
 {'sku': '2042698766637', 'amount': 5},
 {'sku': '2042698766699', 'amount': 1},
 {'sku': '2042698760567', 'amount': 1},
 {'sku': '2042698806401', 'amount': 2},
 {'sku': '2042698760581', 'amount': 1},
 {'sku': '2042698762226', 'amount': 1},
 {'sku': '2042698769423', 'amount': 2},
 {'sku': '2042698768921', 'amount': 2},
 {'sku': '2042698806449', 'amount': 2},


In [7]:
BATCH_SIZE = 1000
REQUEST_LIMIT = 300  # Максимум 300 запросов в минуту
REQUEST_INTERVAL = 60 / REQUEST_LIMIT  # Интервал между запросами

for i in range(0, len(stocks_data), BATCH_SIZE):
    batch = stocks_data[i:i + BATCH_SIZE]

    payload = {"stocks": batch}
    url = WB_API_N_URL.format(warehouseId=WAREHOUSE_ID)
    response = requests.put(url, json=payload, headers=HEADERS)

    if response.status_code == 204:
        print(f"✅ Успешно обновлены остатки для {len(batch)} товаров")
    else:
        print(f"❌ Ошибка при обновлении: {response.text}")

    time.sleep(REQUEST_INTERVAL)  # Задержка для соблюдения лимита API

❌ Ошибка при обновлении: [{"data":[{"sku":"2042698760246","amount":0},{"sku":"2042698760253","amount":0},{"sku":"2042698760291","amount":0},{"sku":"2042698762134","amount":0},{"sku":"2042698760765","amount":0},{"sku":"2042698761311","amount":0},{"sku":"2042698761175","amount":0},{"sku":"2042698761281","amount":0},{"sku":"2042698762981","amount":0},{"sku":"2042698762868","amount":0},{"sku":"2042698762394","amount":0},{"sku":"2042698762141","amount":0},{"sku":"2042698765807","amount":0},{"sku":"2042698765890","amount":0},{"sku":"2042698769058","amount":0},{"sku":"2042698768907","amount":0},{"sku":"2042698769225","amount":0},{"sku":"2042698769102","amount":0}],"code":"CargoWarehouseRestrictionSGTKGTPlus","message":"The selected warehouse is not suitable for goods with the type \"ODC/CD+\". Upload the balances to the warehouse with the label - ODC or CD+"}]
